In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np
import numpy.random as nprand
import pandas as pd
import random
import scipy.stats as spstats
%matplotlib inline

In [ ]:
def get_proposal_map(df):
    """Map proposal names to generic names."""
    proposals = df.columns[3:]
    proposal_map = dict([(proposal, 'prop{}'.format(i + 1)) for i, proposal in enumerate(proposals)])
    proposal_rev_map = dict([('prop{}'.format(i + 1), proposal) for i, proposal in enumerate(proposals)])
    return proposal_map, proposal_rev_map

def load_loomio_score(filename):
    """
    Load results from netdelib:export_results rake task.
    
    Parameters
    ----------
    filename: filename of a TSV of the rake output.
    
    Returns
    -------
    (df, proposal_map, proposal_rev_map)
    df: Pandas dataframe containing stage, participant_id, and proposal scores
    proposal_map: dict mapping original proposal names to generic names, e.g. "prop1"
    proposal_rev_map: dict mapping generic names back to original proposal names
    """
    df = pd.read_csv(filename, delimiter='\t')
    df['participant_id'] = pd.to_numeric(df.participant_id)
    proposal_map, proposal_rev_map = get_proposal_map(df)
    score_map = dict((k, '{}_score'.format(v)) for k, v in proposal_map.items())
    df = df.rename(mapper=score_map, axis='columns')
    return df, proposal_map, proposal_rev_map

class Preference(object):
    """
    Represents a rank-ordered preference.
    
    Parameters
    ----------
    ranked : ranked list of alternatives
    
    """
    
    def __new__(cls, ranked, _cache={}):
        """Load existing object or create new if necessary"""
        try:
            return _cache[ranked]
        except KeyError:
            obj = super(Preference, cls).__new__(cls)
            obj.__init__(ranked)
            _cache[ranked] = obj
            return obj
        
    def __init__ (self, ranked):
        self.ranked = tuple(ranked)
    
    def __hash__ (self):
        return hash(self.ranked)
    
    def __repr__ (self):
        return repr(self.ranked)
    
    def __str__ (self):
        return str(self.ranked)
    
    def __len__ (self):
        return len(self.ranked)
    
    def __getitem__ (self, key):
        return self.ranked[key]
    
    def alternatives (self):
        """
        Return a set of all ranked alternatives.
        """
        return set(self.ranked)
    
    def ranks (self):
        alternatives = sorted(self.alternatives())
        rank_alt = [(rank, alt) for rank, alt in enumerate(self.ranked)]
        rank_alt = sorted(rank_alt, key=lambda x: x[1])
        ranks = [rank for rank, alt in rank_alt]
        return ranks

    
class Profile(object):
    """
    Represents a social preference profile. The profile counts the number ballots expressing
    each `Preference`.
    """
    
    def __init__ (self):
        self.preference_counts = {}
        
    def add(self, preference):
        self.preference_counts[preference] = self.preference_counts.get(preference, 0) + 1
        
    def alternatives (self):
        preference_alternatives = [p.alternatives() for p in self.preference_counts.keys()]
        return set().union(*preference_alternatives)
        
    def counts (self):
        return self.preference_counts.items()
    
    @classmethod
    def from_score(cls, df):
        """Create a profile from a score dataframe.

        Parameters
        ----------
        df: Pandas dataframe with columns: "treatment", "stage", "participant_id", "prop1_score", "prop2_score", ...

        Returns
        -------
        A new Profile instance.
        """
        participant_ids = sorted(set(df.participant_id))
        participant_preferences = {}
        proposals = df.columns[3:]

        # Get preference tuples for each participant
        for pid in participant_ids:
            participant_df = df[df.participant_id == pid]
            try:
                prop_scores = [
                    (p, participant_df[p].to_numpy()[0])
                    for p in proposals]
            except IndexError:
                # Participant did not participate in this round
                continue
            # Sort in order of decreasing score
            
            prop_rank = reversed(sorted(prop_scores, key=lambda x: x[1]))
            ranked = tuple([prop[0:-6] for prop, rank in prop_rank])
            participant_preferences[pid] = Preference(ranked)
            
        # Add preferences to profile
        profile = Profile()
        for pid, preference in participant_preferences.items():
            profile.add(preference)
            
        return profile

    def ballot_count(self):
        return sum([count for preference, count in self.counts()], 0)
    
    def sample_bootstrap(self):
        n = self.ballot_count()
        preferences = list(self.preference_counts.keys())
        # Sampling probabilities for each preference
        p = [self.preference_counts[pref] / n for pref in preferences]
        # Initialize new profile
        bootstrap = Profile()
        # Take n samples with replacement
        count = len(preferences)
        for i in range(n):
            bootstrap.add(preferences[nprand.choice(count, p=p)])
        return bootstrap
            
    def agreement_spearman(self):
        total = 0
        count = 0
        for pref_a, count_a in self.counts():
            for pref_b, count_b in self.counts():
                if pref_a == pref_b:
                    continue
                r, p = spstats.spearmanr(pref_a.ranks(), pref_b.ranks())
                total += count_a * count_b * r
                count += count_a * count_b
        return total / count
    
    def agreement_kendall(self):
        total = 0
        count = 0
        for pref_a, count_a in self.counts():
            for pref_b, count_b in self.counts():
                if pref_a == pref_b:
                    continue
                r, p = spstats.kendalltau(pref_a.ranks(), pref_b.ranks())
                total += count_a * count_b * r
                count += count_a * count_b
        return total / count


In [ ]:
class SocialWelfare(object):
    """
    Base class for social welfare and social choice calculations.
    
    Parameters
    ----------
    profile : ranked-choice voting profile
    """
    
    def __init__(self, profile):
        self.profile = profile
    
    def welfare_score (self):
        """Calculate a social welfare score for each alternative present in profile.
        
        returns a dict mapping alternatives to their scores according to the provided profile.
        """
        raise NotImplemented
    
    def social_preference (self):
        """Calculate the social preference ranking of the alternatives in profile.
        
        returns a tuple of sets listing alternatives in order of decreasing social preference,
            possibly inluding ties.
        """
        score = self.welfare_score()
        ordered_scores = sorted(score.items(), reverse=True, key=lambda x: x[1])
        social_preference = []
        last_score = ordered_scores[0][1]
        current_set = set()
        for alternative, score in ordered_scores:
            # If the score has changed, create a new set of alternatives
            if score == last_score:
                current_set.add(alternative)
            else:
                social_preference.append(current_set)
                current_set = set([alternative])
            last_score = score
        social_preference.append(current_set)
        return social_preference      
    
    def social_choice (self):
        """Calculate the social preference ranking of the alternatives in profile.
        
        returns a set containing the alternative(s) with the highest social welfare score.
        """
        social_preference = self.social_preference()
        return social_preference[0]
    
class Majority(SocialWelfare):
    """Social welfare function for majority vote. An alternative's score is the number
    of preferences in the profile that place the alternative in first place.
    """
    
    def welfare_score (self):
        profile = self.profile
        score = {}
        for pref, count in profile.counts():
            top = pref[0]
            try:
                score[top] += count
            except KeyError:
                score[top] = count
        return score

    
class Borda(SocialWelfare):
    
    def welfare_score (self):
        profile = self.profile
        borda = {}
        for pref, count in profile.counts():
            for i, alternative in enumerate(reversed(pref)):
                try:
                    borda[alternative] += i * count
                except KeyError:
                    borda[alternative] = i * count
        return borda
    
    
import itertools
import networkx as nx

class Tideman(SocialWelfare):
    
    def pairwise_margin (self):
        """Find the win/loss margin for each pair of alternatives.
        
        Returns
        -------
        A dict mapping (a,b) to <prefers(a) - prefers(b)>
        """
        # Create a list of distinct pairs
        alternatives = self.profile.alternatives()
        pairs = set(itertools.product(alternatives, alternatives))
        for alt in alternatives:
            pairs.remove((alt, alt))
        # Initialize margins
        margins = dict((contest, 0) for contest in pairs)
        # Update margins for each preference in the profile
        for pref, count in profile.counts():
            # Each element wins relative to those that come after it
            for win in range(len(pref)):
                for lose in range(win + 1, len(pref)):
                    margins[(pref[win], pref[lose])] += count
        return margins
        
    def social_preference_graph (self):
        profile = self.profile
        margins = self.pairwise_margin()
        ordered_margins = sorted(margins.items(), key=lambda x: x[1], reverse=True)
        G = nx.DiGraph()
        counted = 0
        skipped = 0
        for contest, margin in ordered_margins:
            s, t = contest
            try:
                if nx.has_path(G, t, s):
                    # Skip contest, would create cycle
                    skipped += margin
                    continue
            except nx.NodeNotFound:
                pass
            # Add node to graph
            G.add_edge(s, t)
            counted += margin
        return G, counted, skipped

    def social_preference (self):
        G, counted, skipped = self.social_preference_graph()
        root = next(nx.topological_sort(G))
        result = []
        done = set()
        shell = set([root])
        while len(shell) > 0:
            next_shell = set()
            # Remove alternatives ranked lower than others in shell
            for s in list(shell):
                for t in list(shell):
                    if G.has_edge(s, t):
                        shell.remove(t)
            # Update list of completed alternatives
            done = done | shell
            # Build next shell
            for s in shell:
                next_shell = (next_shell | set(G.successors(s))) - done
            # Add current shell to preference
            result.append(shell)
            shell = next_shell
        return result

    def agreement_tideman (self):
        G, counted, skipped = self.social_preference_graph()
        return counted / (counted + skipped)

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
stage_profiles = [
    Profile.from_score(df_score[df_score.stage == stage])
    for stage in sorted(set(df_score.stage))]

Methods = [Majority, Borda, Tideman]
print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
for stage, profile in enumerate(stage_profiles):
    winners = [repr(M(profile).social_choice()) for M in Methods]
    print(str(stage) + '\t' + '\t'.join(winners))


## Group-level

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')

treatments = {
    1: "Single Group",
    2: "Random Pod",
}

for treatment in [1, 2]:
    print("Treatment: {}".format(treatments[treatment]))
    stage_profiles = [
        Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == treatment)])
        for stage in sorted(set(df_score.stage))]

    Methods = [Majority, Borda, Tideman]
    print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
    for stage, profile in enumerate(stage_profiles):
        winners = [repr(M(profile).social_choice()) for M in Methods]
        print(str(stage) + '\t' + '\t'.join(winners))
    print("")


## Agreement

In [ ]:
class TimeSeriesResult (object):
    
    def __init__ (self):
        # Exact value of the result
        self.value = []

        # Number of samples
        self.counts = []

        # Total of all sample results and squares
        self.totals = []
        self.squares = []

    def add_sample (self, t, v):
        """Add sample v at time t"""
        # Increment count
        try:
            self.counts[t] += 1
        except IndexError:
            self.counts.append(1)
        # Add sample value to total
        try:
            self.totals[t] += v
        except IndexError:
            self.totals.append(v)
        # Add square of sample to sum of squares
        try:
            self.squares[t] += v * v
        except IndexError:
            self.squares.append(v * v)
            
    def add_y (self, v):
        self.value.append(v)
    
    def y (self):
        """Return exact values"""
        return self.value
    
    def mean (self):
        """Calculate mean values"""
        return [self.totals[i] / self.counts[i] for i in range(len(self.totals))]
    
    def var (self):
        """Calculate variance values"""
        m = self.mean()
        n = len(m)
        return [(self.squares[i] - m[i] * m[i]) / self.counts[i] for i in range(n)]
    
    def se (self):
        """Calculate the standard error values"""
        v = self.var()
        n = len(v)
        return [v[i] / math.sqrt(self.counts[i]) for i in range(n)]
    
    def yerr95(self):
        """Calculate 95% confidence intervals"""
        return [sei * 1.96 for sei in self.se()]

In [ ]:
control_profiles = [
    Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == 1)])
    for stage in sorted(set(df_score.stage))]

random_profiles = [
    Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == 2)])
    for stage in sorted(set(df_score.stage))]

bootstrap_runs = 10

kendall_control = TimeSeriesResult()
spearman_control = TimeSeriesResult()
tideman_control = TimeSeriesResult()
print('Single Group')
print('stage\tkendall\tspearman\ttideman')
for stage, profile in enumerate(control_profiles):
    
    kendall_control.add_y(profile.agreement_kendall())
    spearman_control.add_y(profile.agreement_spearman())
    tideman_control.add_y(Tideman(profile).agreement_tideman())
    
    for run in range(bootstrap_runs):
        p = profile.sample_bootstrap()
        kendall_control.add_sample(stage, p.agreement_kendall())
        spearman_control.add_sample(stage, p.agreement_spearman())
        tideman_control.add_sample(stage, Tideman(p).agreement_tideman())
    
    print("{}\t{}\t{}\t{}".format(
        stage,
        kendall_control.y()[stage],
        spearman_control.y()[stage],
        tideman_control.y()[stage]))

    
kendall_random = TimeSeriesResult()
spearman_random = TimeSeriesResult()
tideman_random = TimeSeriesResult()
print('\nRandom Pod')
print('stage\tkendall\tspearman\ttideman')
for stage, profile in enumerate(random_profiles):
    
    kendall_random.add_y(profile.agreement_kendall())
    spearman_random.add_y(profile.agreement_spearman())
    tideman_random.add_y(Tideman(profile).agreement_tideman())
    
    for run in range(bootstrap_runs):
        p = profile.sample_bootstrap()
        kendall_random.add_sample(stage, p.agreement_kendall())
        spearman_random.add_sample(stage, p.agreement_spearman())
        tideman_random.add_sample(stage, Tideman(p).agreement_tideman())
    
    print("{}\t{}\t{}\t{}".format(
        stage,
        kendall_random.y()[stage],
        spearman_random.y()[stage],
        tideman_random.y()[stage]))

In [ ]:
plt.errorbar(range(4), kendall_control.y(), yerr=kendall_control.yerr95(), label='Single Group')
plt.errorbar(range(4), kendall_random.y(), yerr=kendall_random.yerr95(), label='Random Pod')
plt.xticks(range(4))
plt.ylim(0,0.25)
plt.xlabel('Stage')
plt.ylabel('Kendall Correlation')
plt.grid()
plt.legend()
plt.title('Agreement (Kendall)')

In [ ]:
plt.errorbar(range(4), spearman_control.y(), yerr=spearman_random.yerr95(), label='Single Group')
plt.errorbar(range(4), spearman_random.y(), yerr=spearman_random.yerr95(), label='Random Pod')
plt.xticks(range(4))
plt.ylim(0,0.25)
plt.xlabel('Stage')
plt.ylabel('Spearman Correlation')
plt.grid()
plt.legend()
plt.title('Agreement (Spearman)')

In [ ]:
plt.errorbar(range(4), tideman_control.y(), yerr=tideman_control.yerr95(), label='Single Group')
plt.errorbar(range(4), tideman_random.y(), yerr=tideman_control.yerr95(),  label='Random Pod')
plt.xticks(range(4))
plt.xlabel('Stage')
plt.ylabel('Agreement (tideman)')
plt.grid()
plt.legend()
plt.title('Agreement (tideman)')